# セグメント木

蟻本p.153

https://ikatakos.com/pot/programming_algorithm/data_structure/segment_tree

https://algo-logic.info/segment-tree/

https://kujira16.hateblo.jp/entry/2016/12/15/000000

セグメント木 (Segment Tree) は要素列を表現するデータ構造の1つ。  
要素列 $A = a_1 a_2 a_3 \ldots a_n$ に対して、区間 $A_{[l, r)}$ に対する操作を高速に行うことができる。

初めに要素列が用意されていれば、クエリはオンラインであってもよい。  
逆にオフラインの場合はクエリをソートすることで、別のデータ構造を用いたさらなる高速化ができる場合がある。

## Range Minimum Query (RMQ)

要素列をセグメント木に格納すると、区間に対する問い合わせを高速化できる。

- 要素1つの更新: $\mathcal{O}(\log{n})$
- 区間 $[l, r)$ の最小値を求める問い合わせ $\mathcal{O}(\log{n})$  

RMQの実装を示す。操作用のメソッドは0-indexedだが、データを格納する配列は1-indexedとしていることに注意。  
問い合わせ操作には分割統治法を用いている。

Range Minimum Query (RMQ) - [AOJ DSL_2_A](http://judge.u-aizu.ac.jp/onlinejudge/description.jsp?id=DSL_2_A)

In [1]:
class RMQ():

    def __init__(self, size, op=min, init_value=10**8):
        """初期化"""
        self.size = size
        self.op = op
        self.init_value = init_value
        n = 2 ** ((size-1).bit_length())
        treesize = n * 2
        st = [init_value for i in range(treesize)]
        self.st = st
    
    @classmethod
    def from_array(cls, a, op=min, init_value=10**8):
        st = cls(len(a), op=op, init_value=init_value)
        for i, x in enumerate(a):
            st.update(i, x)
        return st
    
    def update(self, key, value):
        """値の更新"""
        offset = len(self.st) // 2
        k = offset + key
        self.st[k] = value
        k >>= 1
        while k > 0:
            self.st[k] = self.op(self.st[k * 2], self.st[k * 2 + 1])
            k >>= 1
    
    def _query(self, a, b, k, l, r):
        """区間[a, b) に対する累積操作
        k: 着目しているノード (1-indexed)
        l: 探索区間 st[l, r) の左端 (0-indexed)
        r: 探索区間 st[l, r) の右端 (0-indexed)
        """
        if r <= a or b <= l:
            return self.init_value
        if a <= l and r <= b:
            return self.st[k]
        mid = (l + r) // 2
        lv = self._query(a, b, k * 2, l, mid)
        rv = self._query(a, b, k * 2 + 1, mid, r)
        return self.op(lv, rv)

    def query(self, a, b):
        """区間[a, b) に対する累積操作"""
        if a > b:
            raise ValueError("a must be less than equal b.")
        return self._query(a, b, k=1, l=0, r=len(self.st)//2)

In [2]:
A = [3, -5, 2, -10, 1, 4, 11]

rmq = RMQ.from_array(A)
print(rmq.query(3, 7))
print([rmq.query(i, i+1) for i in range(len(A))])

-10
[3, -5, 2, -10, 1, 4, 11]


セグメント木による区間に対する操作は、最小値以外の操作でも利用できる。  
具体的には、次の条件を満たせばよい。

- 結合法則が成り立つ $(a \cdot b) \cdot c = a \cdot (b \cdot c)$
- 単位元 $e$ をもつ $a \cdot e = e \cdot a = a$

操作 `op` には二項演算子を、初期値 `init_value` には単位元を指定する。

| クエリ | 操作 | 初期値 |
|:--|:--|:--:|
| 和 | `operator.add` | 0 |
| 積 | `operator.mul` | 1 |
| 最小値 | `min` | +INF |
| 最大値 | `max` | -INF |
| AND | `operator.and` | 1 |
| OR | `operator.and` | 0 |
| XOR | `operator.xor` | 0 |
| GCD | `math.gcd` | 0 |
| LCM | &#x2015; | 1 |

## Range Add Query (RAQ)

区間に対する更新操作を高速化することもできる。

- 区間に対する加算 $\mathcal{O}(\log{n})$
- 値の取得 $\mathcal{O}(\log{n})$

### アルゴリズム

RMQと異なり、区間ノード (内部ノード) には区間全体に加算すべき値を持たせる。

加算処理は次の通り。 根から葉に向かって処理する。

1. ノード $k$ を根とする。
1. 以下の処理を再帰的に行う(*) 。
    1. ノード $k$ が葉ならば、葉ノードに直接加算して終了。
    1. ノード $k$ が区間ノードのとき
        1. $k$ が表す区間 $[l, r)$ が $[a, b)$ に完全に含まれていれば、区間ノードに直接加算して終了。
        1. 部分的に含まれていれば、区間を二分する。子ノード $2k, 2k+1$ をノードとして(*)に戻る。
        1. まったく含まれていなければ、区間を破棄する。

値の取得処理は次の通り。葉から根に向かって処理する。

1. ノード $k$ を葉とする。
1. 初期値 $v=0$ とおく。
1. 以下の処理を再帰的に行う(*) 。
    1. ノードの値を $v$ に加算する。
    1. ノードが根であれば $v$ を出力して終了。
    1. 親ノード $k/2$ をノードとして(*)に戻る。

Range Add Query (RAQ) - [AOJ DSL_2_E](http://judge.u-aizu.ac.jp/onlinejudge/description.jsp?id=DSL_2_E)

In [3]:
class RAQ():

    def __init__(self, size):
        """初期化"""
        self.size = size
        n = 2 ** ((size-1).bit_length())
        treesize = n * 2
        self.st = [0 for i in range(treesize)]
    
    @classmethod
    def from_array(cls, a):
        st = cls(len(a))
        for i, x in enumerate(a):
            st.add(i, i+1, x)
        return st
    
    def _add(self, a, b, value, k, l, r):
        """区間[a, b) に対する加算
        k: 着目しているノード (1-indexed)
        l: 探索区間 st[l, r) の左端 (0-indexed)
        r: 探索区間 st[l, r) の右端 (0-indexed)
        """
        if r <= a or b <= l:
            return
        if l == r - 1:
            self.st[k] += value
            return
        if a <= l and r <= b:
            self.st[k] += value
            return
        mid = (l + r) // 2
        self._add(a, b, value, k * 2, l, mid)
        self._add(a, b, value, k * 2 + 1, mid, r)

    def add(self, a, b, value):
        """区間[a, b) に対する加算"""
        if a > b:
            raise ValueError("a must be less than equal b.")
        n = len(self.st) // 2
        return self._add(a, b, value, k=1, l=0, r=n)

    def get(self, key):
        """値の取得"""
        offset = len(self.st) // 2
        k = offset + key
        v = self.st[k]
        k >>= 1
        while k > 0:
            v += self.st[k]
            k >>= 1
        return v

In [4]:
A = [8, -5, 2, -10, 1, 4, 11]

raq = RAQ.from_array(A)
print([raq.get(i) for i in range(len(A))])
raq.add(2, 5, 5)
print([raq.get(i) for i in range(len(A))])

[8, -5, 2, -10, 1, 4, 11]
[8, -5, 7, -5, 6, 4, 11]
